## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

Импортируем библиотеки для работы с датафреймом и для выполнения математических операций и работы с массивами.

Читаем файл, выводим первые 5 строк.

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('datasets/data.csv')
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Выводим последние 5 строк:

In [2]:
df.tail()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


Получим краткую информацию о данных:

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Проверим количество пропусков:

In [5]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Проверим количество дубликатов:

In [6]:
df.duplicated().sum()

54

Посмотрим на датасет более пристально:

из `df.describe()` видно, что минимальное количество детей равно -1. 

Это странно.

В дальнейшем мы сможем трактовать эту переменную как категориальную, поэтому посмотрим, какие значения и в каком количестве существуют:

In [7]:
df.sort_values('children')['children'].value_counts().sort_index()

-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64

Будем иметь в виду. Потому что -1 – это почти как 1.

А 3, 4, 5  – уже просто много, такие семьи называют многодетными.

20 детей – вообще аномально. Особенно если учесть, что таких семей аж 76 штук, больше, чем с 4 или 5 детьми вместе взятыми.
___

Так же `df.describe()` указал на отрицательные значения в `days_employed`.

Проверим, так ли это, и в каком количестве встречаются такие "странности".

In [8]:
df[df['days_employed'] < 0]['days_employed'].count()

15906

___
Посмотрим, какие варианты `education` нам даны.

Как минимум, в  `df.head()` было видно, что там есть, с чем поработать.

In [9]:
df.sort_values('education')['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

In [10]:
df.sort_values('education_id')['education_id'].value_counts()

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

Более подробно займёмся этим, когда будем лемматизировать.

Например, очевидно, что 4-я категория `education_id` сложилась из четырёх **ученая степень**, одной **Ученая степень** и одной **УЧЕНАЯ СТЕПЕНЬ** в `education`.

___
Проверим, какие сюрпризы ещё готовят нам переменные `family_status`, `gender`, `income_type` и `purpose`.

In [11]:
df.sort_values('family_status')['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [12]:
df.sort_values('gender')['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [13]:
df.sort_values('income_type')['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [14]:
df.sort_values('purpose')['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

### Вывод

Перед нами датасет с 21525 записями.

Проблемы, над которыми надо работать:
- отсутствующие значения в днях стажа и ежемесячном доходе
- отрицательные значения в днях стажа и количестве детей
- есть дубликаты, их 54, но после обработки могут появиться новые
- записи с аномальным количеством детей (20)
- типы данных с плавающей точкой – в стаже и доходе
- одна запись с непонятным гендером: т.к. гендер не участвует в анализе, удалять не будем
- так же странное, но не используемое в анализе: минимальный возраст 0
- уровень образования написан разным регистром, необходимо применить `lowercase()`
- очень много категорий `purpose`, необходимы лемматизация и укрупнение категорий
- дублирующие столбцы `family_status_id` и `education_id`

### Шаг 2. Предобработка данных

### Обработка пропусков

В стобцах `days_employed` и `total_income` имеются пропуски. 

Изучим их более пристально:

In [15]:
print(df[df['days_employed'].isnull()].head(10))

    children  days_employed  dob_years education  education_id  \
12         0            NaN         65   среднее             1   
26         0            NaN         41   среднее             1   
29         0            NaN         63   среднее             1   
41         0            NaN         50   среднее             1   
55         0            NaN         54   среднее             1   
65         0            NaN         21   среднее             1   
67         0            NaN         52    высшее             0   
72         1            NaN         32    высшее             0   
82         2            NaN         50    высшее             0   
83         0            NaN         52   среднее             1   

            family_status  family_status_id gender  income_type  debt  \
12       гражданский брак                 1      M    пенсионер     0   
26        женат / замужем                 0      M  госслужащий     0   
29  Не женат / не замужем                 4      F    

Похоже, что пропуски в этих столбцах присутствуют только вместе.

Убедимся в том, что нет таких строк с пропуском только в `days_employed` или в `total_income`:

In [16]:
str_compare = df['days_employed'].isnull() == df['total_income'].isnull()
print(np.unique(str_compare.values))

[ True]


In [17]:
print('Итого строк с пропусками:', len(df[df['days_employed'].isnull()]))

Итого строк с пропусками: 2174


 Узнаем, какую часть от всех данных занимают пропуски в `days_employed` и `total_income`:

In [18]:
isnull_percentage = len(df[df['days_employed'].isnull()])/len(df['days_employed'])
print("Доля пропущенных значений в 'days_employed': {:.0%}".format(isnull_percentage))

Доля пропущенных значений в 'days_employed': 10%


Это достаточно много, ведь для удаления пропусков без последствий для данных такая доля должна составлять около $1\%$.
***


Такие пропуски дают основание полагать, что эти люди никогда не работали, так как молоды или работали не по ТК РФ. 

Либо произошла некорректная выгрузка данных.
***

Заменим пропуски в `days_employed` и `total_income` на нули, т.к. заполнение средними/медианными значениями может понизить качество данных и ложным образом повысит вероятность выдачи кредита:

In [19]:
df['days_employed'] = df['days_employed'].fillna(0)
df['total_income'] = df['total_income'].fillna(0.)

Проверим, остались ли пропуски:

In [20]:
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Вывод

Такие пропуски – не случайные, они могли быть вызваны спецификой респондентов (не работающих официально).

Заполнение нулём – самое логичное, что можно сделать в этом случае. 

Ведь официально у этих людей нет ни стажа, ни зарплаты.

### Замена типа данных

Поменяем все отрицательные значения `days_employed` на положительные и приведём к целочисленному типу.

Используем метод `astype`, т.к. `to_numeric` служит для конвертации строк и приводит их к числам с плавающей точкой.

In [21]:
df['days_employed'] = np.abs(df['days_employed'])
df['days_employed'] = df['days_employed'].astype('int')

`total_income` тоже будет удобнее наблюдать в целых числах:

In [22]:
df['total_income'] = df['total_income'].astype('int')

Проверим результат:

In [23]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


### Вывод

На данные удобнее смотреть в едином формате, особенно если это `int64`.

Преобразовывать `days_employed` дальше для решения задачи не требуется, т.к. относительно этой величины не требуется анализ зависимостей `debt`.

### Обработка дубликатов

В начале исследования было известно, что в датасете 54 дубликата. 

Поскольку были проведены некоторые действия (заполнение пропусков, приведение к целочисленному типу), узнаем, изменилось ли количество дубликатов.

Перед эти сделаем ещё кое-что: приведём категории образования к нижнему регистру.

In [24]:
df['education'] = df['education'].str.lower()
df.sort_values('education')['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Узнаем, скрывались ли дубли под написанием разными регистрами:

воспользуемся методом `sum()`, применённым к методу `duplicated()`, чтобы видеть не весь список, кто дубликат, а кто - нет, 

а их суммарное количество.

In [25]:
df.duplicated().sum()

71

Ага! На 17 дублей больше.

Есть основания предположить, что первые 54 дубля появились из-за ошибки правил валидации на сервере, 
т.к. там заявки должны проверяться на дубли тв первую очередь.

А дополнительные 17 могли появиться вледствие некорректной интеграции данных.

Очевидно, что в этом случае дублирующие записи отражают реально не существующие объекты.

Поэтому смело удалим их, а точнее сольём 2 одинаковых записи в одну.

____


Удалим дубликаты методом `drop_duplicates()` и при этом сбросим индекс:

In [26]:
df = df.drop_duplicates().reset_index(drop=True)

Проверим, остались ли дубликаты:

In [27]:
df.duplicated().sum()

0

### Вывод

Удалили дублирующиеся записи, т.к. видно, что дублирование является следствием технического сбоя и человеческого фактора/некорректной интеграции данных. 

Очевидно, что в этом случае дубликаты отражают реально не существующие объекты, что нарушает общую логику и репрезентативность данных. 

### Лемматизация

In [28]:
print('Уникальных purpose:', len(df['purpose'].unique()))
print(df['purpose'].unique())

Уникальных purpose: 38
['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


Видно, что все цели состоят из 2 слов или более и прослеживается некоторая закономерность.

Разбить данные на чёткие категории поможет лемматизация.
___

После первого запуска лемматизации было видно, что появились ненужные предлоги 'со', 'на', ' ', '\n'.

Для них надо добавить фильтр **not in**.

In [29]:
from collections import Counter # контейнер для подсчёта числа упоминаний

from pymystem3 import Mystem # библиотека Яндекса для лемматизации
m = Mystem() 

# Попробуем найти самые частовстречаемые слова в леммах и на основе результата определить категории целей кредита
purposes = []

for purpose in df['purpose']:
    lemma = m.lemmatize(purpose)
    purposes += filter(lambda l: l not in [' ', '\n', 'с', 'со', 'на'], lemma)

print(Counter(purposes))

Counter({'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'ремонт': 607, 'подержанный': 486, 'подержать': 478, 'приобретение': 461, 'профильный': 436})


Методом пристального вглядывания выделим 5 основных категорий, а именно:


- недвижимость
- жилье
- автомобиль
- образование
- свадьба
___
Напишем функцию, которая будет справляться лучше пристального взгляда и создадим новый столбец с целями получения кредитного займа.

___
Но сначала создадим список категорий, на которые будем разбивать.

Я оставила категории  **жилье** и **недвижимость**, не соединяя их вместе, т.к. не во всём недвижимом можно жить и не во всём жилье можно остаться недвижимым.

Так же основанием для этого стало их количество: **'недвижимость': 6367**, **'жилье': 4473** – обе категории вполне равноправны.

Если их соединить, будет сильный перевес в сторону недвижимости относительно остальных данных.

In [30]:
purpose_category = [
    'свадьба',
    'недвижимость',
    'жилье',
    'автомобиль',
    'образование'
]

In [31]:
def cat_purpose(string):
    
    lemmas = m.lemmatize(string)
    for lemma in lemmas:
        for category in purpose_category:
            if category in lemma:
                return category

Протестируем написанную выше функцию:

In [32]:
print(cat_purpose('хочу свадьбу'))
print(cat_purpose('жильё недорого'))
print(cat_purpose('избыток недвижимости за МКАДом'))
print(cat_purpose('куплю автомобили дорого'))
print(cat_purpose('высших образований много не бывает (но это не точно)'))

свадьба
жилье
недвижимость
автомобиль
образование


Работает! Единственное число, именительный падеж.

Спасибо разработчикам из Яндекса за такую замечательную библиотеку.

### Вывод

В нашем случае лемматизацию использовать удобнее, чем стемминг. 

Благодаря лемматизации, сразу получаем отдельные слова, сведённые к лемме (именительный падеж, единственное число).

Написанная функция лемматизирует по пересечённым множествам (список лемм, созданный методом пристального взгляда) и список на выходе `m.lemmatize(string)`. На мой взгляд, это эффективнее. А ещё интереснее в реализации.

### Категоризация данных

### Категоризация по цели кредитного займа

Добавляем новый столбец с категориями целей кредитного займа, используя вышенаписанную функцию:

In [33]:
df['purpose_category'] = df['purpose'].apply(cat_purpose)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


### Категоризация по количеству детей

Разберёмся с количеством детей:

ещё в начале проекта увидели аномалии в виде -1 ребёнка и подозрительно большое количество семей (76) с 20 детьми.

После обработки этих аномалий создадим категории по количеству детей.

___
-1 детей заменим на 1, а затем проверим, удалось ли это сделать:

In [34]:
df['children'] = df['children'].replace(-1, 1)

print('Осталось человек с количеством детей -1:', df[df['children'] == -1]['children'].count())

Осталось человек с количеством детей -1: 0


Посмотрим возраст людей с 20 детьми, проверим эти данные на правдоподобность: 

In [35]:
df[df['children'] == 20].groupby('dob_years')['dob_years'].count()

dob_years
0     1
21    1
23    1
24    1
25    1
26    1
27    2
29    2
30    3
31    2
32    2
33    2
34    3
35    2
36    2
37    4
38    1
39    1
40    4
41    2
42    3
43    2
44    2
45    3
46    3
48    1
49    3
50    3
51    1
52    1
53    1
54    1
55    1
56    5
57    1
59    2
60    1
61    1
62    1
64    1
69    1
Name: dob_years, dtype: int64

Иметь до 40 лет 20 детей – это сильное заявление, однако не имеющее ничего общего с правдой.

Скорее всего, при сборе данных допущена ошибка и вместо 2**0** детей имелось в виду **2**.

Заменим 20 на 2:

In [36]:
df['children'] = df['children'].replace(20, 2)

print('Осталось человек с количеством детей 20:', df[df['children'] == 20]['children'].count())

Осталось человек с количеством детей 20: 0


___
Посмотрим, как теперь выглядит статистика по детям в целом:

In [37]:
df.sort_values('children')['children'].value_counts().sort_index()

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

Категоризуем следующим образом:

- 0 детей
- 1 ребёнок
- 2 детей
- 3+ детей – в эту категорию войдут люди с 3, 4, 5 детьми как многодетные

In [38]:
def cat_children(num):
    if num == 0:
        return '0 детей'
    elif num == 1:
        return '1 ребёнок'
    elif num == 2:
        return '2 детей'
    return  '3+ детей'

Создаём новый столбец, применяя вышенаписанную функцию к столбцу `df['children']`:

In [39]:
df['children_category'] = df['children'].apply(cat_children)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,children_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье,1 ребёнок
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,1 ребёнок
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье,0 детей
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,3+ детей
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,0 детей


### Категоризация ежемесячных доходов

У нас нет каких-либо сведений о геопозиции проведенного сбора данных или о местах работы респондентов.

Поделим все данные о ежемесячном доходе на 4 части с помощью квантилей:

In [40]:
df['total_income'].quantile([0.25,0.5,0.75])

0.25     89088.50
0.50    135781.00
0.75    195813.25
Name: total_income, dtype: float64

Категоризуем данные в соответствии с полученными значениями.

Доходы разделим на:

- низкий (< 890088.50)
- средний (890088.50 < ... < 135781.00)
- выше среднего (135781.00 < ... < 195813.25)
- высокий (> 195813.25)

In [41]:
income_q25 = df['total_income'].quantile(0.25)
income_q50 = df['total_income'].quantile(0.5)
income_q75 = df['total_income'].quantile(0.75)

In [42]:
def cat_income(total_income):
    if total_income <= income_q25:
            return 'низкий'
    elif total_income <= income_q50:
            return 'средний'
    elif total_income < income_q75:
            return 'выше среднего'
    return 'высокий'

Создадим новый столбец с категорией дохода:

In [43]:
df['income_category'] = df['total_income'].apply(cat_income)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,children_category,income_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье,1 ребёнок,высокий
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,1 ребёнок,средний
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье,0 детей,выше среднего
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,3+ детей,высокий
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,0 детей,выше среднего


### Категоризация по семейному положению

У нас есть целых 5 уникальных значений.
Сформируем из них 2 категории:

- **в браке** – женат / замужем, гражданский брак
- **не в браке** – не женат / не замужем, в разводе, вдовец / вдова

In [44]:
def cat_family_status(string):
    if string == 'женат / замужем':
        return 'в браке'
    elif string == 'гражданский брак':
        return 'в браке'
    return 'не в браке'

Создадим новый столбец по данным `df['family_status']` и применённой к нему вышенаписанной функции:

In [45]:
df['family_status_category'] = df['family_status'].apply(cat_family_status)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,children_category,income_category,family_status_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье,1 ребёнок,высокий,в браке
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,1 ребёнок,средний,в браке
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье,0 детей,выше среднего,в браке
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,3+ детей,высокий,в браке
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,0 детей,выше среднего,в браке


### Вывод

Категоризовали все данные, необходимые для дальнейшего анализа, представив, что если бы респонденты заполняли анкету, то в этой анкете требовалось бы только поставить галочки напротив нужного варианта.

Для каждого из четырёх новых столбцов была выбрана своя стратегия категоризации:
- цели кредита - с помощью лемматизации
- количество детей - укрупнение категорий
- ежемесячный доход - разбиение на квантили
- семейное положение - укрупнение категорий

Так же для каждого нового столбца была написана функция, которая брала данные из столбца-оригинала и преобразовывала их в категориальный тип.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [46]:
df.pivot_table(index='children_category', values='debt', aggfunc='mean')

,debt
children_category,
0 детей,0.075438
1 ребёнок,0.091658
2 детей,0.094925
3+ детей,0.081579


### Вывод

Прямой зависимости между количеством детей и погашением кредита без задолженностей ней.

Однако:

- Семьи без детей выплачивают кредит чаще, чем семьи с детьми.

- Из семей с детьми лучше всего с выплатой кредитов справляются многодетные. Возможно, им на помощь приходит материнский капитал или другие выплаты от государства.

- Хуже всего выплачивают кредит семьи с 2 детьми.
___

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [47]:
df.pivot_table(index='family_status_category', values='debt', aggfunc='mean')

,debt
family_status_category,
в браке,0.079988
не в браке,0.085012


### Вывод

Люди, не состоящие в браке, хуже справляются с выплатой кредита примерно на 0.5%.
___

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [48]:
df.pivot_table(index='income_category', values='debt', aggfunc='mean')

,debt
income_category,
высокий,0.071402
выше среднего,0.089689
низкий,0.078113
средний,0.085400


### Вывод

- Лучше всего возвращают кредит люди с **высоким** доходом

- Люди с доходом **выше среднего** возвращают кредит хуже всех остальных категорий. Это может быть связано с тем, что они "не рассчитали" свои возможности, когда брали кредит или предоставили ложную справку о доходах

- Люди со **средним** доходом чуть реже, чем люди с доходом **выше среднего**, возвращают задолженность. Это может быть связано с теми же причинами

- **Низкий** доход не сказывается негативно на вероятности возвращения кредита. По крайней мере, его возвращают чаще, чем с более бОльшими доходами (кроме категории высокий)

___
- Как разные цели кредита влияют на его возврат в срок?

In [49]:
df.pivot_table(index='purpose_category', values='debt', aggfunc='mean')

,debt
purpose_category,
автомобиль,0.093590
жилье,0.069058
недвижимость,0.074634
образование,0.092200
свадьба,0.080034


### Вывод

Можно увидеть, что чаще всего задолженности по кредитам возникают в случае покупки автомобиля и образовательных услуг.

Чуть реже (примерно на 10-15 человек из 100) появляются задолженности по кредиту для оплаты свадьбы.

Наиболее часто без задолженностей обходятся покупка недвижимости и жилья.

### Шаг 4. Общий вывод

Исходя из предыдущих выводов, следующие пункты могут оказаться полезными при создании модели кредитного скоринга: 
безопаснее всего выдавать кредиты на покупку жилья, недвижимости или свадьбу – по ним меньшее количество задолжностей.

Для более точного ответа нужно рассматривать все факторы в совокупности:
- выгоднее выдавать кредит семьям без детей или, если с детьми, то многодетным;
- людям с низким доходом или, наоборот, с высоким;
- обращать внимание на семейное положение: лучше выдать кредит человеку "в браке".

Это может быть портретом идеального заёмщика, однако для полной и более точной модели необходимо задействовать трудовой стаж, уровень образования и, возможно, пол заёмщика. 
Эти данные могут помочь найти новые зависимости и построить новые гипотезы о "портрете идеального заёмщика".


### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.